In [19]:
import torch
from onnx_tf.backend import prepare
import onnx
import tensorflow as tf

In [20]:
loaded = torch.jit.load('7x7/best.pt')
# loaded = torch.jit.load('5x5/best.pt')
# loaded = torch.jit.load('src/dualnet5x5_v2.pt')
# loaded = torch.jit.load('src/dualnet5x5_se.pt')
# loaded = torch.jit.load('src/dualnet7x7_se.pt')
print(loaded)

RecursiveScriptModule(
  original_name=DualNet
  (conv1): RecursiveScriptModule(original_name=Conv2d)
  (bn1): RecursiveScriptModule(original_name=BatchNorm2d)
  (relu): RecursiveScriptModule(original_name=ReLU)
  (layers): RecursiveScriptModule(
    original_name=Sequential
    (0): RecursiveScriptModule(
      original_name=BasicBlock
      (relu): RecursiveScriptModule(original_name=ReLU)
      (bn1): RecursiveScriptModule(original_name=BatchNorm2d)
      (conv1): RecursiveScriptModule(original_name=Conv2d)
      (bn2): RecursiveScriptModule(original_name=BatchNorm2d)
      (conv2): RecursiveScriptModule(original_name=Conv2d)
      (bn3): RecursiveScriptModule(original_name=BatchNorm2d)
      (conv3): RecursiveScriptModule(original_name=Conv2d)
      (downsample): RecursiveScriptModule(original_name=Sequential)
      (seblock): RecursiveScriptModule(
        original_name=SEBlock
        (conv1): RecursiveScriptModule(original_name=Conv2d)
        (relu): RecursiveScriptModule(origi

In [21]:
loaded.eval()
bisze = 7
dummy_input = torch.randn((3, 12, bisze, bisze))
torch.onnx.export(loaded, dummy_input, "dualnet.onnx", example_outputs=torch.rand((3, bisze*bisze+2)), verbose=True)

graph(%x.1 : Float(3, 12, 7, 7, strides=[588, 49, 7, 1], requires_grad=0, device=cpu),
      %layers.0.bn1.running_var : Float(128, strides=[1], requires_grad=0, device=cuda:0),
      %layers.0.bn1.running_mean : Float(128, strides=[1], requires_grad=0, device=cuda:0),
      %layers.0.bn1.bias : Float(128, strides=[1], requires_grad=0, device=cuda:0),
      %layers.0.bn1.weight : Float(128, strides=[1], requires_grad=0, device=cuda:0),
      %layers.0.conv1.weight : Float(32, 128, 1, 1, strides=[128, 1, 1, 1], requires_grad=0, device=cuda:0),
      %layers.0.bn2.running_var : Float(32, strides=[1], requires_grad=0, device=cuda:0),
      %layers.0.bn2.running_mean : Float(32, strides=[1], requires_grad=0, device=cuda:0),
      %layers.0.bn2.bias : Float(32, strides=[1], requires_grad=0, device=cuda:0),
      %layers.0.bn2.weight : Float(32, strides=[1], requires_grad=0, device=cuda:0),
      %layers.0.conv2.weight : Float(32, 32, 3, 3, strides=[288, 9, 3, 1], requires_grad=0, device=cud

In [22]:
tf.autograph.experimental.do_not_convert
onnx_model = onnx.load('dualnet.onnx')

tf_rep = prepare(onnx_model, device='gpu')
tf_rep.export_graph('model.pb')

INFO:tensorflow:Assets written to: model.pb\assets


In [23]:
!cd train
!saved_model_cli show --dir ./model.pb --all

�w�肳�ꂽ�p�X��������܂���B



MetaGraphDef with tag-set: 'serve' contains the following SignatureDefs:

signature_def['__saved_model_init_op']:
  The given SavedModel SignatureDef contains the following input(s):
  The given SavedModel SignatureDef contains the following output(s):
    outputs['__saved_model_init_op'] tensor_info:
        dtype: DT_INVALID
        shape: unknown_rank
        name: NoOp
  Method name is: 

signature_def['serving_default']:
  The given SavedModel SignatureDef contains the following input(s):
    inputs['x.1'] tensor_info:
        dtype: DT_FLOAT
        shape: (3, 12, 7, 7)
        name: serving_default_x.1:0
  The given SavedModel SignatureDef contains the following output(s):
    outputs['output_0'] tensor_info:
        dtype: DT_FLOAT
        shape: (3, 51)
        name: PartitionedCall:0
  Method name is: tensorflow/serving/predict

Defined Functions:
  Function Name: '__call__'
        Named Argument #1
          x.1

  Function Name: 'gen_tensor_dict'


In [24]:

!tensorflowjs_converter --input_format=tf_saved_model --output_node_names='output_0' --saved_model_tags=serve model.pb js_model --quantize_uint8

Writing weight file js_model\model.json...
weight PartitionedCall/onnx_tf_prefix_Constant_0 with shape (4,) and dtype int64 was auto converted to the type int32
weight PartitionedCall/Where with shape (0, 1) and dtype int64 was auto converted to the type int32
weight PartitionedCall/Cast with shape (1,) and dtype int64 was auto converted to the type int32
weight PartitionedCall/GatherV2 with shape (0,) and dtype int64 was auto converted to the type int32
weight PartitionedCall/zeros with shape () and dtype int64 was auto converted to the type int32


2021-12-19 12:13:51.069486: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN)to use the following CPU instructions in performance-critical operations:  AVX AVX2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-12-19 12:13:53.748737: I tensorflow/core/grappler/devices.cc:78] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0 (Note: TensorFlow was not compiled with CUDA or ROCm support)
2021-12-19 12:13:53.749569: I tensorflow/core/grappler/clusters/single_machine.cc:356] Starting new session
2021-12-19 12:13:54.284433: I tensorflow/core/grappler/optimizers/meta_optimizer.cc:816] Optimization results for grappler item: graph_to_optimize
2021-12-19 12:13:54.284696: I tensorflow/core/grappler/optimizers/meta_optimizer.cc:818]   function_optimizer: Graph size after: 1785 nodes (1600), 1733 edges (1549), time = 98.003ms.
2021-12-19 12:13